In [ ]:
from exchangelib import Account, Credentials, OAuth2Credentials, DELEGATE
from requests_oauthlib import OAuth2Session
import webbrowser
import os
from http.server import HTTPServer, BaseHTTPRequestHandler
import urllib.parse
import sys

# Thông tin ứng dụng Azure AD
client_id = "your_client_id"  # Thay bằng Client ID từ Azure AD
client_secret = "your_client_secret"  # Thay bằng Client Secret (nếu cần)
tenant_id = "your_tenant_id"  # Thay bằng Tenant ID (hoặc "common" cho Outlook cá nhân)
redirect_uri = "http://localhost:8000"
scope = ["https://outlook.office365.com/EWS.AccessAsUser.All"]  # Quyền truy cập EWS
email_address = "your_email@outlook.com"  # Thay bằng email của bạn

# Thư mục lưu email
save_folder = "downloaded_emails"
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Biến để lưu access token
access_token = None

# Tạo server đơn giản để nhận callback từ trình duyệt
class OAuthCallbackHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        global access_token
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(b"Authentication successful! You can close this window.")

        # Lấy code từ URL callback
        query = urllib.parse.urlparse(self.path).query
        params = urllib.parse.parse_qs(query)
        code = params.get("code", [None])[0]

        if code:
            # Tạo OAuth session để lấy token
            oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
            token = oauth.fetch_token(
                token_url=f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token",
                client_secret=client_secret,
                code=code,
                include_client_id=True
            )
            access_token = token["access_token"]
            print("Access token received!")

# Hàm chính để tải email
def download_emails():
    try:
        # Tạo OAuth session
        oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
        authorization_url, state = oauth.authorization_url(
            f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/authorize"
        )

        # Mở trình duyệt để đăng nhập
        print("Mở trình duyệt để đăng nhập...")
        webbrowser.open(authorization_url)

        # Khởi động server để nhận callback
        server = HTTPServer(("localhost", 8000), OAuthCallbackHandler)
        server.handle_request()  # Xử lý một yêu cầu (callback từ trình duyệt)

        if not access_token:
            print("Không lấy được access token. Kiểm tra lại quá trình đăng nhập.")
            sys.exit(1)

        # Cấu hình thông tin xác thực với access token
        credentials = OAuth2Credentials(
            access_token=access_token,
            client_id=client_id,
            client_secret=client_secret,
            tenant_id=tenant_id
        )

        # Kết nối tới tài khoản Outlook
        account = Account(
            primary_smtp_address=email_address,
            credentials=credentials,
            autodiscover=True,
            access_type=DELEGATE
        )

        # Duyệt qua tất cả email trong Inbox
        for item in account.inbox.all().order_by('-datetime_received'):
            subject = item.subject.replace('/', '_').replace('\\', '_').replace(':', '_')
            if not subject:
                subject = "No Subject"
            file_name = f"{save_folder}/{subject}_{item.datetime_received.strftime('%Y%m%d_%H%M%S')}.eml"
            with open(file_name, 'wb') as f:
                f.write(item.mime_content)
            print(f"Đã lưu: {file_name}")

        print("Tải email hoàn tất!")

    except Exception as e:
        print(f"Lỗi xảy ra: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    download_emails()

UnauthorizedError: Invalid credentials for https://outlook.office365.com/EWS/Exchange.asmx